The approach is very simple. We will drop any column which can not be converted to numerical feature in astraight forward way and consider only the following columns for prediction
1. region
2. city
3. parent_category_name
4. category_name
5. param_1
6. param_2
7. param_3
8. price
9. item_seq_number
10. user_type

We will just ignore the rest of the columns for now
A lot of things can be and should be improved over this e.g. hyperparameter optimization and most importantly **we should definitely use title and description to improve**.
You may consider my code a bit untidy. But I will improve this as I move forward

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

First, lets load the training data

In [3]:
data_raw = pd.read_csv('../input/train.csv')

Drop anything that we'll not be needing for now

In [4]:
data_raw = data_raw.drop('item_id', axis=1).drop('user_id', axis=1).drop('title', axis=1).drop('description', axis=1)\
        .drop('activation_date', axis=1).drop('user_type', axis=1).drop('image', axis=1).drop('image_top_1', axis=1)

**Make all those feature mentioned categorical now. Because there are managable number of unique values in those columns**

In [6]:
region_set = data_raw['region'].unique()
def convert_region(x):
    tp = np.where(region_set == x['region'])
    return tp[0][0]
data_raw['region'] = data_raw.apply(convert_region, axis=1)

In [7]:
city_set = data_raw['city'].unique()
def convert_city(x):
    tp = np.where(city_set == x['city'])
    return tp[0][0]
data_raw['city'] = data_raw.apply(convert_city, axis=1)

In [9]:
cat_set = data_raw['category_name'].unique()
def convert_cat(x):
    tp = np.where(cat_set == x['category_name'])
    return tp[0][0]
data_raw['category_name'] = data_raw.apply(convert_cat, axis=1)

In [8]:
p1_set = data_raw['param_1'].unique()
def convert_p1(x):
    tp = np.where(p1_set == x['param_1'])
    try:
        return tp[0][0]
    except Exception:
        return 0
data_raw['param_1'] = data_raw.apply(convert_p1, axis=1)

In [10]:
p2_set = data_raw['param_2'].unique()
def convert_p2(x):
    tp = np.where(p2_set == x['param_2'])
    try:
        return tp[0][0]
    except Exception:
        return 0
data_raw['param_2'] = data_raw.apply(convert_p2, axis=1)

In [11]:
p3_set = data_raw['param_3'].unique()
def convert_p3(x):
    tp = np.where(p3_set == x['param_3'])
    try:
        return tp[0][0]
    except Exception:
        return 0
data_raw['param_3'] = data_raw.apply(convert_p3, axis=1)

In [12]:
parent_category_name_set = data_raw['parent_category_name'].unique()
def convert_parent_category_name(x):
    tp = np.where(parent_category_name_set == x['parent_category_name'])
    return tp[0][0]
data_raw['parent_category_name'] = data_raw.apply(convert_parent_category_name, axis=1)

Now our data is ready. We will prepare our `x_train` and `y_train`

In [14]:
x_train = data_raw.drop('deal_probability', axis=1).as_matrix()
y_train = data_raw['deal_probability'].as_matrix()

In [15]:
import xgboost as xgb

**Lets train our simple xgb model**

In [16]:
xgdmat=xgb.DMatrix(x_train, y_train)
params={'seed':0,'colsample_bytree':0.8,'objective':'reg:linear','max_depth':24,'min_child_weight':24}
final_gb=xgb.train(params,xgdmat)

**Reading test data**

In [17]:
test_df = pd.read_csv('../input/test.csv')

Dropping the columns we don't need

In [ ]:
test_df = test_df.drop('user_id', axis=1).drop('title', axis=1).drop('description', axis=1)\
        .drop('activation_date', axis=1).drop('user_type', axis=1).drop('image', axis=1).drop('image_top_1', axis=1)

**Make all columns categorical instead of string, as done in training set**

In [18]:
def convert_region(x):
    tp = np.where(region_set == x['region'])
    return tp[0][0]
# creating region catagory as numerical feature
test_df['region'] = test_df.apply(convert_region, axis=1)

In [19]:
def convert_city(x):
    tp = np.where(city_set == x['city'])
    try:
        return tp[0][0]
    except Exception:
        return 0
test_df['city'] = test_df.apply(convert_city, axis=1)

In [20]:
def convert_cat(x):
    tp = np.where(cat_set == x['category_name'])
    try:
        return tp[0][0]
    except Exception:
        return 0
test_df['category_name'] = test_df.apply(convert_cat, axis=1)

In [21]:
def convert_p1(x):
    tp = np.where(p1_set == x['param_1'])
    try:
        return tp[0][0]
    except Exception:
        return 0
test_df['param_1'] = test_df.apply(convert_p1, axis=1)

In [22]:
def convert_p2(x):
    tp = np.where(p2_set == x['param_2'])
    try:
        return tp[0][0]
    except Exception:
        return 0
test_df['param_2'] = test_df.apply(convert_p2, axis=1)

In [23]:
def convert_p3(x):
    tp = np.where(p3_set == x['param_3'])
    try:
        return tp[0][0]
    except Exception:
        return 0
test_df['param_3'] = test_df.apply(convert_p3, axis=1)

In [24]:
def convert_parent_category_name(x):
    tp = np.where(parent_category_name_set == x['parent_category_name'])
    try:
        return tp[0][0]
    except Exception:
        return 0
test_df['parent_category_name'] = test_df.apply(convert_parent_category_name, axis=1)

**prepare `x_test`**

In [28]:
x_test = test_df.drop('item_id', axis=1).as_matrix()

In [29]:
tesdmat=xgb.DMatrix(x_test)
y_pred=final_gb.predict(tesdmat)

In [31]:
y_pred[y_pred<0] = 0

At this point our prediction is ready. Lets prepare the output dataframe

In [45]:
result = pd.DataFrame({ 'deal_probability': y_pred, 'item_id': test_df['item_id']})

**All done. Lets create our submission file**

In [46]:
result.to_csv('submission.csv', index=False)